In [23]:
import numpy as np
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen

In [24]:
Cleveland_data_URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data'
np.set_printoptions(threshold=np.nan)
names = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','heartdisease']
heartDisease = pd.read_csv(urlopen(Cleveland_data_URL), names = names)
heartDisease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heartdisease
0,28,1,2,130,132,0,2,185,0,0.0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0


In [25]:
del heartDisease['ca']
del heartDisease['slope']
del heartDisease['thal']
del heartDisease['oldpeak']
heartDisease = heartDisease.replace('?',np.nan)
heartDisease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,heartdisease
0,28,1,2,130,132,0,2,185,0,0
1,29,1,2,120,243,0,0,160,0,0
2,29,1,2,140,NaN,0,0,170,0,0
3,30,0,1,170,237,0,1,170,0,0
4,31,0,2,100,219,0,1,150,0,0


In [26]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator,BayesianEstimator 


In [27]:
model = BayesianModel([('age','trestbps'),('age','fbs'),('sex','trestbps'),('sex','trestbps'),('fbs','heartdisease'),
                       ('exang','trestbps'),('trestbps','heartdisease'),('heartdisease','restecg'),('heartdisease','thalach'),
                      ('heartdisease','chol')])
model.fit(heartDisease,estimator=MaximumLikelihoodEstimator)


In [28]:
print(model.get_cpds('age'))
print(model.get_cpds('fbs'))
print(model.get_cpds('chol'))
print(model.get_cpds('heartdisease'))

╒═════════╤════════════╕
│ age(28) │ 0.00383142 │
├─────────┼────────────┤
│ age(29) │ 0.00383142 │
├─────────┼────────────┤
│ age(30) │ 0.00383142 │
├─────────┼────────────┤
│ age(31) │ 0.00766284 │
├─────────┼────────────┤
│ age(32) │ 0.0153257  │
├─────────┼────────────┤
│ age(33) │ 0.00766284 │
├─────────┼────────────┤
│ age(34) │ 0.0153257  │
├─────────┼────────────┤
│ age(35) │ 0.0191571  │
├─────────┼────────────┤
│ age(36) │ 0.0191571  │
├─────────┼────────────┤
│ age(37) │ 0.0306513  │
├─────────┼────────────┤
│ age(38) │ 0.0191571  │
├─────────┼────────────┤
│ age(39) │ 0.0344828  │
├─────────┼────────────┤
│ age(40) │ 0.0191571  │
├─────────┼────────────┤
│ age(41) │ 0.0383142  │
├─────────┼────────────┤
│ age(42) │ 0.0268199  │
├─────────┼────────────┤
│ age(43) │ 0.0421456  │
├─────────┼────────────┤
│ age(44) │ 0.0268199  │
├─────────┼────────────┤
│ age(45) │ 0.0229885  │
├─────────┼────────────┤
│ age(46) │ 0.045977   │
├─────────┼────────────┤
│ age(47) │ 0.0344828  │


In [29]:
model.get_independencies()

(age _|_ sex, exang)
(age _|_ sex | exang)
(age _|_ restecg, thalach, chol | heartdisease)
(age _|_ exang | sex)
(age _|_ sex, exang | fbs)
(age _|_ restecg, chol | heartdisease, thalach)
(age _|_ thalach, chol | restecg, heartdisease)
(age _|_ restecg, thalach, chol | heartdisease, exang)
(age _|_ sex | exang, fbs)
(age _|_ restecg, thalach, chol | heartdisease, trestbps)
(age _|_ restecg, heartdisease, thalach, chol | fbs, trestbps)
(age _|_ restecg, thalach | heartdisease, chol)
(age _|_ restecg, thalach, chol | heartdisease, sex)
(age _|_ restecg, thalach, chol | heartdisease, fbs)
(age _|_ exang | sex, fbs)
(age _|_ chol | restecg, heartdisease, thalach)
(age _|_ restecg, chol | heartdisease, thalach, exang)
(age _|_ restecg, chol | heartdisease, thalach, trestbps)
(age _|_ restecg, heartdisease, chol | thalach, fbs, trestbps)
(age _|_ restecg | heartdisease, thalach, chol)
(age _|_ restecg, chol | heartdisease, thalach, sex)
(age _|_ restecg, chol | heartdisease, thalach, fbs)
(a

In [30]:
from pgmpy.inference import VariableElimination 
heartInfer = VariableElimination(model)
q = heartInfer.query(variables=['heartdisease'],evidence={'age':28})
print(q['heartdisease'])

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.5770 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.4230 │
╘════════════════╧═════════════════════╛


C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]


In [31]:
q = heartInfer.query(variables=['heartdisease'], evidence={'chol': 100})
print(q['heartdisease']) 

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.5510 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.4490 │
╘════════════════╧═════════════════════╛


C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:598: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]
C:\ProgramData\Anaconda3\lib\site-packages\pgmpy\factors\discrete\DiscreteFactor.py:586: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
